In [1]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter


!pip install efficientnet_pytorch
from zipfile import ZipFile
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
import torch.optim as optim
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models


drive.mount('/content/drive')

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=d75d68ed9474a50bd6f6aec7effbb7fd062a1516e7ccea9bd500f7096f3d83ca
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch
Mounted at /content/drive


In [2]:
import zipfile


with zipfile.ZipFile('/content/drive/MyDrive/flowerimages.zip', 'r') as zip_ref:
    zip_ref.extractall('unzipped_images_flower')


In [3]:
!pip install Pillow


In [4]:
from PIL import Image
import os


unzipped_folder = '/content/unzipped_images_flower/flowers'


subdirectories = [d for d in os.listdir(unzipped_folder) if os.path.isdir(os.path.join(unzipped_folder, d))]

seen = set()
for flower_category in subdirectories:
    folder_path = os.path.join(unzipped_folder, flower_category)
    image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]

    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        image = Image.open(image_path)
        width, height = image.size
        seen.add((width, height))

# lets check if all images have the same dimensions
print(len(seen))


272


In [5]:
from PIL import Image
import os

unzipped_folder = '/content/unzipped_images_flower/flowers'
output_folder = '/content/resized_images_flower'


os.makedirs(output_folder, exist_ok=True)

subdirectories = [d for d in os.listdir(unzipped_folder) if os.path.isdir(os.path.join(unzipped_folder, d))]

seen = set()
for flower_category in subdirectories:
    input_folder = os.path.join(unzipped_folder, flower_category)
    output_category_folder = os.path.join(output_folder, flower_category)


    os.makedirs(output_category_folder, exist_ok=True)

    image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]

    for image_file in image_files:
        input_image_path = os.path.join(input_folder, image_file)
        output_image_path = os.path.join(output_category_folder, image_file)

        image = Image.open(input_image_path)

        # Resize the image
        target_size = (456, 456)
        image = image.resize(target_size, Image.ANTIALIAS)

        # Convert the image to grayscale
        # image = image.convert('L')


        image.save(output_image_path)

        width, height = image.size
        seen.add((width, height))




<ipython-input-5-9eb77c6bfed0>:30: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image = image.resize(target_size, Image.ANTIALIAS)


In [6]:
# from sklearn.model_selection import train_test_split


# data = []

# for flower_category in subdirectories:
#     category_folder = os.path.join(output_folder, flower_category)
#     image_files = [f for f in os.listdir(category_folder) if f.endswith('.jpg')]
#     label = subdirectories.index(flower_category)  # Assign a label based on the folder name

#     for image_file in image_files:
#         image_path = os.path.join(category_folder, image_file)
#         data.append((image_path, label))


# train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


from sklearn.model_selection import train_test_split


max_images_per_class =40

data = []

for flower_category in subdirectories:
    category_folder = os.path.join(output_folder, flower_category)
    image_files = [f for f in os.listdir(category_folder) if f.endswith('.jpg')]
    label = subdirectories.index(flower_category)  # Assign a label based on the folder name

    # Limit the number of images to max_images_per_class
    image_files = image_files[:max_images_per_class]

    for image_file in image_files:
        image_path = os.path.join(category_folder, image_file)
        data.append((image_path, label))

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


In [7]:
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, label = self.data[idx]
        image = Image.open(image_path)
        if self.transform:
            image = self.transform(image)
        return image, label

train_dataset = CustomDataset(train_data, transform=transform)
test_dataset = CustomDataset(test_data, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [ ]:

model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=16)


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

max_epoch = [10, 20, 30]
for e in max_epoch:
    # Training
    for epoch in range(e):
        print(epoch)
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 50 == 49:
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0

    # Testing
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print("for number of epochs being:", e)
    print('Accuracy of the network on the test images: %d %%' % (
        100 * correct / total))

# Save the trained model
torch.save(model.state_dict(), "/content/drive/MyDrive/images/flower_efficient30")


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 287MB/s]


Loaded pretrained weights for efficientnet-b0
0
1
2
3
4
5
6
7
8
9
for number of epochs being: 10
Accuracy of the network on the test images: 67 %
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
for number of epochs being: 20
Accuracy of the network on the test images: 79 %
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
for number of epochs being: 30
Accuracy of the network on the test images: 83 %
